### Registration and masks

**Learning outcomes:**
- How to apply registration to an image and its corresponding mask using ants.

In [1]:
%matplotlib inline

import os
from helpers import *

import ants
# noinspection PyPep8Naming
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.5.4
SimpleITK version = 2.4.1


In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

project folder = /home/younes/PycharmProjects/MRI-preprocessing-techniques


In [3]:
raw_examples = [
    'brain-lesion_T1w.nii.gz',
]

#### Raw image

In [4]:
raw_example = raw_examples[0]
raw_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', raw_example)
raw_img_ants = ants.image_read(raw_img_path, reorient='IAL')

explore_3D_array(arr=raw_img_ants.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [5]:
mask_example = add_suffix_to_filename(raw_example, suffix='mask')
mask_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', mask_example)
mask_img_ants = ants.image_read(mask_img_path, reorient='IAL')

explore_3D_array_with_mask_contour(
    arr=raw_img_ants.numpy(),
    mask=mask_img_ants.numpy()
)

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

#### Template image

In [6]:
template_img_path = os.path.join(BASE_DIR, 'assets', 'templates', 'mni_icbm152_t1_tal_nlin_sym_09a.nii')
template_img_ants = ants.image_read(template_img_path, reorient='IAL')

explore_3D_array(arr = template_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [7]:
print('\t\tRAW IMG')
print(raw_img_ants)

print('\t\tTEMPLATE IMG')
print(template_img_ants)


		RAW IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (192, 192, 160)
	 Spacing    : (1.25, 1.25, 1.2)
	 Origin     : (98.1114, -149.1525, -129.5975)
	 Direction  : [-0.  0. -1.  0.  1.  0.  1.  0.  0.]

		TEMPLATE IMG
ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]



#### Registration

In [8]:
transformation = ants.registration(
    fixed=template_img_ants,
    moving=raw_img_ants, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [0x748fd8ef2768,0x748fd908b808,1] -m mattes[0x748fd8ef2768,0x748fd908b808,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0x748fd8ef2768,0x748fd908b808,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [/tmp/tmpi52lc7ly,0x748fd8ef1e88,0x748fd8ef30c8] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1
All_Command_lines_OK
Using single precision for computations.
The composite transform comprises the following transforms (in order): 
  1. Center of mass alignment using fixed image: 0x748fd8ef2768 and moving image: 0x748fd908b808 (type = Euler3DTransform)
  Reading mask(s).
    Registration stage 0
      No fixed mask
      No moving mask
    Registration stage 1
      No fixed mask
      No moving mask
  number of levels = 4
  number of levels = 3
  fixed image: 0x748fd8ef2768
  moving image: 0x748fd908b808
  fixed image: 0x748fd8ef2768
  moving image: 0x748fd908b808
D

In [9]:
print(transformation)

{'warpedmovout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [-0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'warpedfixout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (192, 192, 160)
	 Spacing    : (1.25, 1.25, 1.2)
	 Origin     : (98.1114, -149.1525, -129.5975)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'fwdtransforms': ['/tmp/tmpi52lc7ly1Warp.nii.gz', '/tmp/tmpi52lc7ly0GenericAffine.mat'], 'invtransforms': ['/tmp/tmpi52lc7ly0GenericAffine.mat', '/tmp/tmpi52lc7ly1InverseWarp.nii.gz']}


In [10]:
registered_img_ants = transformation['warpedmovout']

explore_3D_array(arr=registered_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [11]:
out_folder =  os.path.join(BASE_DIR, 'assets', 'preprocessed')
out_folder = os.path.join(out_folder, raw_example.split('.')[0]) # create folder with name of the raw file
os.makedirs(out_folder, exist_ok=True) # create folder if not exists

out_filename = add_suffix_to_filename(raw_example, suffix='registered')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR):])
print(out_path[len(BASE_DIR):])

/assets/raw_examples/brain-lesion_T1w.nii.gz
/assets/preprocessed/brain-lesion_T1w/brain-lesion_T1w_registered.nii.gz


In [12]:
registered_img_ants.to_file(out_path)

#### Move raw mask from native space.

In [13]:
registered_mask_img_ants = ants.apply_transforms(
    moving=mask_img_ants,
    fixed=transformation['warpedmovout'],
    transformlist=transformation['fwdtransforms'],
    verbose=True
)

['-d', '3', '-i', '0x748fd8525bc8', '-o', '0x748fd8527368', '-r', '0x748fd8ef2cc8', '-n', 'linear', '-t', '/tmp/tmpi52lc7ly1Warp.nii.gz', '-t', '/tmp/tmpi52lc7ly0GenericAffine.mat']
Using double precision for computations.
Input scalar image: 0x748fd8525bc8
Could not create ImageIO for the input file, assuming dimension = 3 and scalar pixel type
Reference image: 0x748fd8ef2cc8
The composite transform comprises the following transforms (in order): 
  1. /tmp/tmpi52lc7ly0GenericAffine.mat (type = AffineTransform)
  2. /tmp/tmpi52lc7ly1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: LinearInterpolateImageFunction
Output warped image: 0x748fd8527368


In [14]:
explore_3D_array_with_mask_contour(
    arr=registered_img_ants.numpy(),
    mask=registered_mask_img_ants.numpy()
)

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [15]:
out_filename = add_suffix_to_filename(mask_example, suffix='registered')
out_path = os.path.join(out_folder, out_filename)

print(out_path[len(BASE_DIR):])

/assets/preprocessed/brain-lesion_T1w/brain-lesion_T1w_mask_registered.nii.gz


In [16]:
registered_mask_img_ants.to_file(out_path)